In [ ]:
from rich import print
import wikipedia as wiki

In [ ]:
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError, WikipediaException

def set_wiki_language(lang: str = "en") -> None:
    """
    设置维基百科 API 的语言。

    Args:
        lang (str): 语言代码 (例如, "en" 代表英语, "zh" 代表中文)。默认为 "en"。

    Raises:
        WikipediaException: 如果设置语言时发生 API 错误。
    """
    try:
        wikipedia.set_lang(lang)
        print(f"Wikipedia language set to: {lang}")
    except Exception as e:
        raise WikipediaException(f"Error setting Wikipedia language to '{lang}': {e}")

def search_wiki_pages(query: str, results: int = 5) -> list[str]:
    """
    根据查询词搜索维基百科页面标题。

    Args:
        query (str): 搜索的关键词。
        results (int): 返回的最大结果数量。默认为 5。

    Returns:
        list[str]: 匹配的维基百科页面标题列表。

    Raises:
        WikipediaException: 如果搜索过程中发生 API 错误。
    """
    try:
        return wikipedia.search(query, results=results)
    except Exception as e:
        raise WikipediaException(f"Error searching Wikipedia for '{query}': {e}")

def get_wiki_page_summary(title: str, sentences: int = 0, auto_suggest: bool = True, redirect: bool = True) -> str | None:
    """
    获取指定维基百科页面的摘要。

    Args:
        title (str): 维基百科页面的确切标题或搜索词。
        sentences (int): 返回摘要的句子数量。0 表示返回第一段。默认为 0。
        auto_suggest (bool): 如果找不到精确匹配，是否自动建议页面。默认为 True。
        redirect (bool): 是否自动处理重定向。默认为 True。

    Returns:
        str | None: 页面的纯文本摘要。如果页面不存在或有歧义，则返回 None 或引发异常（取决于错误类型）。

    Raises:
        PageError: 如果页面不存在 (当 auto_suggest=False 时)。
        DisambiguationError: 如果标题导致消歧义页面。
        WikipediaException: 如果获取摘要过程中发生其他 API 错误。
    """
    try:
        # 使用 page() 获取页面对象，以便更好地处理歧义和不存在的情况
        page_obj = wikipedia.page(title, auto_suggest=auto_suggest, redirect=redirect)
        if sentences > 0:
            return wikipedia.summary(title, sentences=sentences, auto_suggest=auto_suggest, redirect=redirect)
        else:
            # 默认返回第一段作为摘要
            return page_obj.summary # 或者使用 wikipedia.summary(title, sentences=0)
    except PageError:
        print(f"PageError: The page '{title}' does not match any pages.")
        return None
    except DisambiguationError as e:
        print(f"DisambiguationError: '{title}' may refer to:")
        for i, option in enumerate(e.options[:5]): # 最多显示5个选项
            print(f"  {i+1}. {option}")
        # 可以选择在这里返回选项列表或引发异常
        # raise e # 重新引发异常，让调用者处理
        return f"DisambiguationError: '{title}' may refer to: {', '.join(e.options[:5])}..."
    except Exception as e:
        raise WikipediaException(f"Error getting summary for '{title}': {e}")


def get_wiki_page_content(title: str, auto_suggest: bool = True, redirect: bool = True) -> str | None:
    """
    获取指定维基百科页面的完整纯文本内容。

    Args:
        title (str): 维基百科页面的确切标题或搜索词。
        auto_suggest (bool): 如果找不到精确匹配，是否自动建议页面。默认为 True。
        redirect (bool): 是否自动处理重定向。默认为 True。

    Returns:
        str | None: 页面的完整纯文本内容。如果页面不存在或有歧义，则返回 None。

    Raises:
        PageError: 如果页面不存在 (当 auto_suggest=False 时)。
        DisambiguationError: 如果标题导致消歧义页面。
        WikipediaException: 如果获取内容过程中发生其他 API 错误。
    """
    try:
        page_obj = wikipedia.page(title, auto_suggest=auto_suggest, redirect=redirect)
        return page_obj.content
    except PageError:
        print(f"PageError: The page '{title}' does not match any pages.")
        return None
    except DisambiguationError as e:
        print(f"DisambiguationError: '{title}' may refer to: {e.options}")
        # 可以选择在这里返回选项列表或引发异常
        return f"DisambiguationError: '{title}' may refer to: {', '.join(e.options[:5])}..."
    except Exception as e:
        raise WikipediaException(f"Error getting content for '{title}': {e}")

def get_wiki_page_metadata(title: str, metadata_keys: list[str] | None = None, auto_suggest: bool = True, redirect: bool = True) -> dict | None:
    """
    获取指定维基百科页面的元数据。

    Args:
        title (str): 维基百科页面的确切标题或搜索词。
        metadata_keys (list[str] | None): 需要获取的元数据键列表。
            可能的键包括: 'title', 'pageid', 'url', 'content', 'summary',
            'images', 'references', 'links', 'sections', 'parent_id',
            'revision_id', 'categories', 'coordinates', 'html'.
            如果为 None，则尝试获取所有可用的元数据。默认为 None。
        auto_suggest (bool): 如果找不到精确匹配，是否自动建议页面。默认为 True。
        redirect (bool): 是否自动处理重定向。默认为 True。

    Returns:
        dict | None: 包含所请求元数据的字典。如果页面不存在或有歧义，则返回 None。

    Raises:
        PageError: 如果页面不存在 (当 auto_suggest=False 时)。
        DisambiguationError: 如果标题导致消歧义页面。
        WikipediaException: 如果获取元数据过程中发生其他 API 错误。
        AttributeError: 如果请求了页面对象不支持的元数据键。
    """
    try:
        page_obj = wikipedia.page(title, auto_suggest=auto_suggest, redirect=redirect)
        metadata = {}
        available_keys = [
            'title', 'pageid', 'url', 'content', 'summary', 'images',
            'references', 'links', 'sections', 'parent_id', 'revision_id',
            'categories', 'coordinates'
            # 'html' is a method, handle separately if needed
        ]

        keys_to_fetch = metadata_keys if metadata_keys is not None else available_keys

        for key in keys_to_fetch:
            try:
                if hasattr(page_obj, key):
                    metadata[key] = getattr(page_obj, key)
                elif key == 'html': # 特殊处理 html() 方法
                     metadata[key] = page_obj.html()
                else:
                    print(f"Warning: Metadata key '{key}' not directly available on page object for '{title}'.")
            except Exception as e:
                print(f"Warning: Could not retrieve metadata key '{key}' for '{title}': {e}")
                metadata[key] = None # 或者标记为错误

        return metadata

    except PageError:
        print(f"PageError: The page '{title}' does not match any pages.")
        return None
    except DisambiguationError as e:
        print(f"DisambiguationError: '{title}' may refer to: {e.options}")
        return f"DisambiguationError: '{title}' may refer to: {', '.join(e.options[:5])}..." # 返回错误信息或 None
    except AttributeError as e:
        raise AttributeError(f"Invalid metadata key requested: {e}")
    except Exception as e:
        raise WikipediaException(f"Error getting metadata for '{title}': {e}")

def get_random_wiki_page_title(pages: int = 1) -> str | list[str]:
    """
    获取一个或多个随机维基百科文章的标题。

    Args:
        pages (int): 要获取的随机页面数量。默认为 1。

    Returns:
        str | list[str]: 如果 pages=1，返回单个标题字符串；否则返回标题列表。

    Raises:
        WikipediaException: 如果获取随机页面时发生 API 错误。
    """
    try:
        return wikipedia.random(pages=pages)
    except Exception as e:
        raise WikipediaException(f"Error getting random Wikipedia page(s): {e}")

# == 示例用法 ==
if __name__ == '__main__':
    try:
        # 设置语言为中文
        set_wiki_language("zh")

        # 搜索页面
        query = "北京"
        search_results = search_wiki_pages(query, results=3)
        print(f"\nSearch results for '{query}': {search_results}")

        if search_results:
            page_title = search_results[0] # 使用第一个搜索结果

            # 获取摘要
            summary = get_wiki_page_summary(page_title, sentences=2)
            if summary:
                print(f"\nSummary for '{page_title}':\n{summary}")

            # 获取内容 (可能很长，谨慎打印)
            # content = get_wiki_page_content(page_title)
            # if content:
            #     print(f"\nContent for '{page_title}' (first 500 chars):\n{content[:500]}...")

            # 获取特定元数据
            metadata = get_wiki_page_metadata(page_title, metadata_keys=['pageid', 'url', 'categories', 'coordinates'])
            if metadata:
                print(f"\nMetadata for '{page_title}':")
                for key, value in metadata.items():
                    print(f"  {key}: {value}")

        # 处理歧义页面示例
        print("\nTesting Disambiguation:")
        summary_dis = get_wiki_page_summary("苹果", auto_suggest=False) # auto_suggest=False 更容易触发
        if isinstance(summary_dis, str) and summary_dis.startswith("DisambiguationError"):
             print(summary_dis)
        elif summary_dis:
             print(f"Summary for '苹果': {summary_dis}")


        # 处理页面不存在示例
        print("\nTesting Page Not Found:")
        summary_404 = get_wiki_page_summary("一个绝对不存在的维基页面标题12345", auto_suggest=False)
        if summary_404 is None:
            print("Page not found as expected.")

        # 获取随机页面
        random_title = get_random_wiki_page_title()
        print(f"\nRandom Wikipedia page title: {random_title}")
        random_titles = get_random_wiki_page_title(pages=2)
        print(f"Random Wikipedia page titles: {random_titles}")

    except WikipediaException as e:
        print(f"\nAn error occurred: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

In [ ]:
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError, WikipediaException
import json # 用于元数据函数返回 JSON 字符串


def search_wiki_pages(query: str, results: int = 5, lang: str = "en") -> list[str] | str:
    """
    根据查询词搜索维基百科页面标题。

    设计用于 LLM Function Calling。成功时返回标题列表，失败时返回错误信息字符串。

    Args:
        query (str): 搜索的关键词。
        results (int): 返回的最大结果数量。默认为 5。
        lang (str): 维基百科语言代码 (例如, "en" 代表英语, "zh" 代表中文)。默认为 "en"。

    Returns:
        list[str] | str: 如果成功，返回匹配的维基百科页面标题列表。
                         如果发生错误（例如 API 问题），返回一个描述错误的字符串。
    """
    try:
        wikipedia.set_lang(lang)
        search_results = wikipedia.search(query, results=results)
        return search_results
    except Exception as e:
        # 返回错误信息字符串而不是抛出异常
        return f"Wikipedia API Error: Failed to search for '{query}' in language '{lang}'. Details: {e}"

def get_wiki_page_summary(title: str, sentences: int = 0, auto_suggest: bool = True, redirect: bool = True, lang: str = "en") -> str:
    """
    获取指定维基百科页面的摘要。

    设计用于 LLM Function Calling。成功时返回页面摘要，失败时返回错误信息字符串。

    Args:
        title (str): 维基百科页面的确切标题或搜索词。
        sentences (int): 返回摘要的句子数量。0 表示返回第一段。默认为 0。
        auto_suggest (bool): 如果找不到精确匹配，是否自动建议页面。默认为 True。
        redirect (bool): 是否自动处理重定向。默认为 True。
        lang (str): 维基百科语言代码 (例如, "en" 代表英语, "zh" 代表中文)。默认为 "en"。

    Returns:
        str: 如果成功，返回页面的纯文本摘要。
             如果页面未找到 (PageError)，返回 "Error: Page not found for title '{title}'."。
             如果标题导致消歧义 (DisambiguationError)，返回 "Error: Disambiguation - '{title}' may refer to: [options list]."。
             如果发生其他 API 错误，返回 "Error: Could not retrieve summary for '{title}'. Details: [error details]."。
    """
    try:
        wikipedia.set_lang(lang)
        # 尝试直接获取摘要，如果失败则捕获异常
        summary_text = wikipedia.summary(title, sentences=sentences, auto_suggest=auto_suggest, redirect=redirect)
        return summary_text
    except PageError:
        return f"Error: Page not found for title '{title}' in language '{lang}'."
    except DisambiguationError as e:
        options = ", ".join(e.options[:5]) # 最多显示5个选项
        return f"Error: Disambiguation - '{title}' in language '{lang}' may refer to: {options}..."
    except Exception as e:
        return f"Error: Could not retrieve summary for '{title}' in language '{lang}'. Details: {e}"


def get_wiki_page_content(title: str, auto_suggest: bool = True, redirect: bool = True, lang: str = "en") -> str:
    """
    获取指定维基百科页面的完整纯文本内容。

    设计用于 LLM Function Calling。成功时返回页面内容，失败时返回错误信息字符串。

    Args:
        title (str): 维基百科页面的确切标题或搜索词。
        auto_suggest (bool): 如果找不到精确匹配，是否自动建议页面。默认为 True。
        redirect (bool): 是否自动处理重定向。默认为 True。
        lang (str): 维基百科语言代码 (例如, "en" 代表英语, "zh" 代表中文)。默认为 "en"。

    Returns:
        str: 如果成功，返回页面的完整纯文本内容。
             如果页面未找到 (PageError)，返回 "Error: Page not found for title '{title}'."。
             如果标题导致消歧义 (DisambiguationError)，返回 "Error: Disambiguation - '{title}' may refer to: [options list]."。
             如果发生其他 API 错误，返回 "Error: Could not retrieve content for '{title}'. Details: [error details]."。
    """
    try:
        wikipedia.set_lang(lang)
        page_obj = wikipedia.page(title, auto_suggest=auto_suggest, redirect=redirect)
        return page_obj.content
    except PageError:
        return f"Error: Page not found for title '{title}' in language '{lang}'."
    except DisambiguationError as e:
        options = ", ".join(e.options[:5])
        return f"Error: Disambiguation - '{title}' in language '{lang}' may refer to: {options}..."
    except Exception as e:
        return f"Error: Could not retrieve content for '{title}' in language '{lang}'. Details: {e}"

def get_wiki_page_metadata(title: str, metadata_keys: list[str] | None = None, auto_suggest: bool = True, redirect: bool = True, lang: str = "en") -> str:
    """
    获取指定维基百科页面的元数据，并以 JSON 字符串形式返回。

    设计用于 LLM Function Calling。成功时返回包含元数据的 JSON 字符串，失败时返回错误信息字符串。

    Args:
        title (str): 维基百科页面的确切标题或搜索词。
        metadata_keys (list[str] | None): 需要获取的元数据键列表。
            可能的键包括: 'title', 'pageid', 'url', 'content', 'summary',
            'images', 'references', 'links', 'sections', 'parent_id',
            'revision_id', 'categories', 'coordinates', 'html'.
            如果为 None，则尝试获取常用元数据 ('title', 'pageid', 'url', 'categories', 'summary')。默认为 None。
        auto_suggest (bool): 如果找不到精确匹配，是否自动建议页面。默认为 True。
        redirect (bool): 是否自动处理重定向。默认为 True。
        lang (str): 维基百科语言代码 (例如, "en" 代表英语, "zh" 代表中文)。默认为 "en"。

    Returns:
        str: 如果成功，返回一个包含所请求元数据的 JSON 格式字符串。
             如果页面未找到 (PageError)，返回 "Error: Page not found for title '{title}'."。
             如果标题导致消歧义 (DisambiguationError)，返回 "Error: Disambiguation - '{title}' may refer to: [options list]."。
             如果请求了无效的元数据键 (AttributeError)，返回 "Error: Invalid metadata key requested: [key name]."。
             如果发生其他 API 错误，返回 "Error: Could not retrieve metadata for '{title}'. Details: [error details]."。
    """
    try:
        wikipedia.set_lang(lang)
        page_obj = wikipedia.page(title, auto_suggest=auto_suggest, redirect=redirect)
        metadata = {}

        default_keys = ['title', 'pageid', 'url', 'categories', 'summary']
        keys_to_fetch = metadata_keys if metadata_keys is not None else default_keys

        for key in keys_to_fetch:
            try:
                if hasattr(page_obj, key):
                    # 特殊处理可能返回非 JSON 兼容类型的属性 (虽然此列表中的大部分是安全的)
                    value = getattr(page_obj, key)
                    # 确保值是 JSON 可序列化的 (简单类型通常没问题)
                    metadata[key] = value
                elif key == 'html': # 特殊处理 html() 方法
                     metadata[key] = page_obj.html() # HTML 字符串是安全的
                else:
                    # 记录无法直接获取的键，但不视为错误
                    metadata[key] = f"Warning: Key '{key}' not directly available or failed to retrieve."
            except Exception as e:
                # 捕获获取单个属性时可能发生的错误
                metadata[key] = f"Warning: Could not retrieve metadata key '{key}'. Details: {e}"

        # 将字典转换为 JSON 字符串
        return json.dumps(metadata, ensure_ascii=False, indent=2) # indent 用于可读性，LLM 可能不需要

    except PageError:
        return f"Error: Page not found for title '{title}' in language '{lang}'."
    except DisambiguationError as e:
        options = ", ".join(e.options[:5])
        return f"Error: Disambiguation - '{title}' in language '{lang}' may refer to: {options}..."
    except AttributeError as e: # 虽然内部尝试捕获，但 page() 本身可能失败
        return f"Error: Invalid metadata key requested or attribute access issue for '{title}'. Details: {e}"
    except Exception as e:
        return f"Error: Could not retrieve metadata for '{title}' in language '{lang}'. Details: {e}"

def get_random_wiki_page_title(pages: int = 1, lang: str = "en") -> str | list[str]:
    """
    获取一个或多个随机维基百科文章的标题。

    设计用于 LLM Function Calling。成功时返回标题或标题列表，失败时返回错误信息字符串。

    Args:
        pages (int): 要获取的随机页面数量。默认为 1。
        lang (str): 维基百科语言代码 (例如, "en" 代表英语, "zh" 代表中文)。默认为 "en"。

    Returns:
        str | list[str]: 如果 pages=1 且成功，返回单个标题字符串。
                         如果 pages > 1 且成功，返回标题字符串列表。
                         如果发生错误，返回一个描述错误的字符串。
    """
    try:
        wikipedia.set_lang(lang)
        random_titles = wikipedia.random(pages=pages)
        return random_titles
    except Exception as e:
        return f"Error: Could not retrieve random Wikipedia page(s) in language '{lang}'. Details: {e}"

# == 示例用法 ==
if __name__ == '__main__':
    # 搜索页面 (中文)
    query = "人工智能"
    print(f"\nSearching for '{query}' in Chinese...")
    search_results = search_wiki_pages(query, results=3, lang="zh")
    print(f"Search Results: {search_results}") # 可能是列表或错误字符串

    # 获取摘要 (英文) - 假设搜索结果是列表且第一个是 'Artificial intelligence'
    page_title_en = "Artificial intelligence"
    print(f"\nGetting summary for '{page_title_en}' in English...")
    summary = get_wiki_page_summary(page_title_en, sentences=2, lang="en")
    print(f"Summary:\n{summary}") # 可能是摘要或错误字符串

    # 获取内容 (中文) - 假设搜索结果是列表且第一个是 '人工智能'
    if isinstance(search_results, list) and search_results:
        page_title_zh = search_results[0]
        print(f"\nGetting content for '{page_title_zh}' in Chinese...")
        content = get_wiki_page_content(page_title_zh, lang="zh")
        if not content.startswith("Error:"):
            print(f"Content (first 200 chars):\n{content[:200]}...")
        else:
            print(content) # 打印错误信息
    else:
        print("\nSkipping content retrieval due to search error or no results.")


    # 获取元数据 (中文) - '北京'
    print(f"\nGetting metadata for '北京' in Chinese...")
    metadata_json = get_wiki_page_metadata("北京", metadata_keys=['pageid', 'url', 'categories', 'coordinates', 'invalid_key'], lang="zh")
    print(f"Metadata (JSON):\n{metadata_json}") # 可能是 JSON 字符串或错误字符串

    # 处理歧义页面示例 (英文)
    print("\nTesting Disambiguation (English):")
    summary_dis = get_wiki_page_summary("Apple", auto_suggest=False, lang="en")
    print(f"Result for 'Apple': {summary_dis}") # 应该返回 Disambiguation 错误字符串

    # 处理页面不存在示例 (英文)
    print("\nTesting Page Not Found (English):")
    summary_404 = get_wiki_page_summary("A non existent wikipedia page title 12345xyz", auto_suggest=False, lang="en")
    print(f"Result for non-existent page: {summary_404}") # 应该返回 Page not found 错误字符串

    # 获取随机页面 (英文)
    print("\nGetting random page title (English):")
    random_title = get_random_wiki_page_title(lang="en")
    print(f"Random title: {random_title}") # 可能是标题或错误字符串

## Google Search

In [ ]:
from googlesearch import search, SearchResult


search_result = search(
    term="OpenAI", 
    num_results=10,
    lang="en",
    advanced=True,
    safe=
    )

In [ ]:
for item in search_results:
    print(item)

## PDFDEAL (doc2x SDK)

In [1]:
from pdfdeal import Doc2X

from dotenv import find_dotenv, load_dotenv
import os

_ = load_dotenv(find_dotenv())


client = Doc2X(
    apikey=os.getenv("DOC2X_API_KEY"),
    debug=True
)


success, failed, flag = client.pdf2file(
    pdf_file="toolrl.pdf",
    output_path="./output",
    output_format="docx",
)

print(success)
print(failed)
print(flag)

2025-04-23 14:56:19,333 - pdfdeal.doc2x - INFO - Uploading toolrl.pdf...
2025-04-23 14:56:23,561 - pdfdeal.doc2x - INFO - Uploading successful for toolrl.pdf with uid 0196616e-ea6c-7104-8ba6-c0fc57ab426b
2025-04-23 14:56:25,153 - pdfdeal.doc2x - INFO - Processing 0196616e-ea6c-7104-8ba6-c0fc57ab426b : 0%
2025-04-23 14:56:30,451 - pdfdeal.doc2x - INFO - Processing 0196616e-ea6c-7104-8ba6-c0fc57ab426b : 19%
2025-04-23 14:56:35,640 - pdfdeal.doc2x - INFO - Processing 0196616e-ea6c-7104-8ba6-c0fc57ab426b : 40%
2025-04-23 14:56:42,662 - pdfdeal.doc2x - INFO - Processing 0196616e-ea6c-7104-8ba6-c0fc57ab426b : 96%
2025-04-23 14:56:47,230 - pdfdeal.doc2x - INFO - Parsing successful for toolrl.pdf with uid 0196616e-ea6c-7104-8ba6-c0fc57ab426b
2025-04-23 14:56:47,231 - pdfdeal.doc2x - INFO - Converting 0196616e-ea6c-7104-8ba6-c0fc57ab426b to docx...
2025-04-23 14:56:49,868 - pdfdeal.doc2x - INFO - Converting 0196616e-ea6c-7104-8ba6-c0fc57ab426b docx file...
2025-04-23 14:56:54,404 - pdfdeal.doc2

['./output/toolrl.docx']
[{'error': '', 'path': ''}]
False
